# Power Outages

**Name(s)**: Ryan Doh and Jerry Wu

**Website Link**: 

## Code

In [2]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [162]:
df = pd.read_csv('outage - Masterdata.csv')
df.columns = df.iloc[4].values
#formatting the dataframe to only include columns and not white space from the excel file
df = df.drop([0, 1, 2,3, 4,5], axis = 0).set_index('OBS').drop(columns = ['variables'])
#Creating outage start and restoration time and storing as time stamps, while dropping unnecessary columns
#that were used to created outage start and outage restoration
df['OUTAGE.START'] =  (df['OUTAGE.START.DATE'] + " " + df['OUTAGE.START.TIME']).apply(pd.Timestamp)
df['OUTAGE.RESTORATION'] = (df['OUTAGE.RESTORATION.DATE'] + " " + df['OUTAGE.RESTORATION.TIME']).apply(pd.Timestamp)
df = df.drop(columns = ['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'])
df['POPDEN_URBAN'] = df['POPDEN_URBAN'].astype(float)
df['POPDEN_RURAL'] = df['POPDEN_RURAL'].astype(float)
df['OUTAGE.DURATION'] = df['OUTAGE.DURATION'].astype(float)
df['YEAR'] = df['YEAR'].astype(int)
#replaces the nan values with np.NaN because they were object types 
df['CLIMATE.CATEGORY'] = df['CLIMATE.CATEGORY'].replace('nan', np.NaN)
df.head()

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,NaN,...,2279.0,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983,2011-07-01 17:00:00,2011-07-03 20:00:00
2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,vandalism,...,2279.0,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983,2014-05-11 18:38:00,2014-05-11 18:39:00
3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,heavy wind,...,2279.0,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983,2010-10-26 20:00:00,2010-10-28 22:00:00
4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,thunderstorm,...,2279.0,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983,2012-06-19 04:30:00,2012-06-20 23:00:00
5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,NaN,...,2279.0,1700.5,18.2,2.14,0.6,91.59266587,8.407334131,5.478742983,2015-07-18 02:00:00,2015-07-19 07:00:00


### Framing the Problem

In [5]:
# Our prediction problem will be a Regression problem, We are going to create a model that will predict the Outage Duration (in minutes)
# of each major power outage. We chose outage duration because we want to see if we could create a model that can accurately 
#predict the length of each major power outage without using time and based solely on other factors related to the outage.
# The metric that we are using to evaluate our model is Root Mean Squared Error (RMSE) and we chose this over Mean Squared Error
# (MSE) and R^2 because R^2 can be artificially increased due to overfitting, whereas RMSE is less likely to be increased. The
# RMSE also takes the square root of the the mean squared error and creates more interpretable error and is less susceptible to
# outliers

### Baseline Model

In [113]:
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [163]:
#df[['YEAR', 'MONTH', 'U.S._STATE', 'NERC.REGION', 'CLIMATE.REGION',\
 #          'CLIMATE.CATEGORY', 'POPPCT_URBAN', 'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC',\
  #         'POPDEN_RURAL', 'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND','PCT_WATER_TOT', 'PCT_WATER_INLAND']]

df = df[['MONTH', 'U.S._STATE', 'NERC.REGION', 'CLIMATE.REGION', 'CLIMATE.CATEGORY', 'POPDEN_URBAN', 'POPDEN_RURAL',\
        'OUTAGE.DURATION']].dropna()
# NEEEEDDDD TO DROP ALL ROWS W/ NAN 
# Can OneHotEncode these categories 'NERC.REGION', 'CLIMATE.REGION', 'CLIMATE.CATEGORY'
# Label encoding for U.S._STATE
# Month probably doesn't need to be changed but also look out for NaN
# POPDEN_URBAN(log scale / sqrt transformation) POPDEN_RURAL (Quantile transformer)
y = df['OUTAGE.DURATION']
X = df.drop('OUTAGE.DURATION', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

coltrans = ColumnTransformer([
    ('OneHotcols', OneHotEncoder(), ['MONTH']),
    ('Numerical', FunctionTransformer(lambda x:x), ['POPDEN_URBAN' , 'POPDEN_RURAL'])
])
pl = Pipeline([('trans', coltrans), 
               ('line', LinearRegression())
              ])

pl.fit(X_train, y_train)
np.sqrt(np.mean((y_test - pl.predict(X_test)) ** 2))

7067.1617337894195

### Final Model

In [175]:
#k-fold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
pipes = {
    'total_bill only': Pipeline([
        ('trans', ColumnTransformer(
            [('keep', FunctionTransformer(lambda x: x), ['POPDEN_URBAN'])], 
            remainder='drop')), 
        ('lin-reg', LinearRegression())
    ]),
    'total_bill + size': Pipeline([
        ('trans', ColumnTransformer(
            [('keep', FunctionTransformer(lambda x: x), ['POPDEN_URBAN', 'POPDEN_RURAL'])], 
            remainder='drop')), 
        ('lin-reg', LinearRegression())
    ]),
    'total_bill + size + OHE smoker': Pipeline([
        ('trans', ColumnTransformer(
            [('keep', FunctionTransformer(lambda x: x), ['POPDEN_URBAN', 'POPDEN_RURAL']),
             ('ohe', OneHotEncoder(), ['MONTH'])], 
            remainder='drop')), 
        ('lin-reg', LinearRegression())
    ]),
    'Climate region': Pipeline([
        ('trans', ColumnTransformer(
            [('keep', FunctionTransformer(lambda x: x), ['POPDEN_URBAN', 'POPDEN_RURAL']),
             ('ohe', OneHotEncoder(), ['MONTH', 'CLIMATE.REGION'])], 
            remainder='drop')), 
        ('lin-reg', LinearRegression())
    ]),

}

pipe_df = pd.DataFrame()

for pipe in pipes:
    errs = cross_val_score(pipes[pipe], X_train, y_train,
                           cv=5, scoring='neg_root_mean_squared_error')
    pipe_df[pipe] = -errs
    
pipe_df.index = [f'Fold {i}' for i in range(1, 6)]
pipe_df.index.name = 'Validation Fold'
pipe_df.mean().idxmin()

'Climate region'

In [180]:
errs_df = pd.DataFrame()

for d in range(1, 26):
    pl = Pipeline([('poly', PolynomialFeatures(d)), ('lin-reg', LinearRegression())])
    
    # The `scoring` argument is used to specify that we want to compute the RMSE; 
    # the default is R^2. It's called "neg" RMSE because, 
    # by default, sklearn likes to "maximize" scores, and maximizing -RMSE is the same
    # as minimizing RMSE.
    errs = cross_val_score(pl, df[['POPDEN_URBAN']], df['OUTAGE.DURATION'], 
                           cv=5, scoring='neg_root_mean_squared_error')
    errs_df[f'Deg {d}'] = -errs # Negate to turn positive (sklearn computed negative RMSE).
    
errs_df.index = [f'Fold {i}' for i in range(1, 6)]
errs_df.index.name = 'Validation Fold'
errs_df

,Deg 1,Deg 2,Deg 3,Deg 4,Deg 5,Deg 6,Deg 7,Deg 8,Deg 9,Deg 10,...,Deg 16,Deg 17,Deg 18,Deg 19,Deg 20,Deg 21,Deg 22,Deg 23,Deg 24,Deg 25
Validation Fold,,,,,,,,,,,,,,,,,,,,,
Fold 1,9096.068434,9087.423960,9076.251840,9070.296970,9042.403514,9025.706097,9023.088714,9030.300470,9042.584982,9056.190855,...,9089.229482,9088.985687,9088.908796,9087.998732,9089.052477,9090.084281,9091.048960,9091.921586,9092.692495,9093.362263
Fold 2,5182.897184,5217.542460,5189.513400,5156.523674,5153.115465,5160.765089,5176.308236,5192.238652,5201.915213,5202.349745,...,5170.607103,5170.326304,5170.408747,5172.352775,5173.171228,5174.017402,5174.845181,5175.622151,5176.330183,5176.962450
Fold 3,3701.979142,3713.540118,3684.266920,3803.194279,3788.865312,3761.602320,3728.316762,3699.541633,3683.760511,3681.331701,...,3707.790097,3708.499597,3705.766607,3706.019886,3706.552583,3707.216077,3707.900001,3708.535175,3709.086103,3709.540694
Fold 4,6986.259560,7151.767425,7171.825819,7191.794470,7076.775333,6898.337421,6745.919439,6696.437389,6709.691285,6707.634557,...,6949.967046,6966.857616,6972.800887,6976.192309,6948.714937,6955.535895,6961.577719,6966.570341,6970.509231,6973.518290
Fold 5,2867.958011,2851.478813,2837.703173,3001.361201,2895.766879,2804.969318,2750.634950,2731.158236,2738.460146,2762.107213,...,2973.585187,2849.588161,2864.646066,2889.021335,2924.870863,2974.960503,3042.737680,3132.390003,3248.878526,3397.937780


In [165]:
# TODO

coltrans = ColumnTransformer([
    ('OneHotcols', OneHotEncoder(), ['NERC.REGION', 'CLIMATE.REGION', 'CLIMATE.CATEGORY']),
    ('LogScale', FunctionTransformer(np.sqrt), ['POPDEN_URBAN']),
    ('Quant_transformer', FunctionTransformer(np.log), ['POPDEN_RURAL']),
])
pl = Pipeline([('trans', coltrans), 
               ('line', LinearRegression())
              ])

pl.fit(X_train, y_train)
np.sqrt(np.mean((y_test - pl.predict(X_test)) ** 2))

6872.288055499312

### Fairness Analysis

In [ ]:
# TODO